<img src="https://drive.google.com/uc?export=view&id=1FsTa2YzWwxY-HBkd2EOXAlkPJ1zpikWv" alt="drawing" width="150"/>


<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1VBn7nKlruxCPLHH_UuD8B2pkvKP7JLRd" alt="drawing" width="800"/>

</br>

<img src="https://drive.google.com/uc?export=view&id=1uF_4x1YqLuS3M0mlh3NrMrEUHvjBMBFw" alt="drawing" width="800"/>
</p>

#Control Cinemático de Manipuladores
* Diego Quiroz Velasquez
* Pontificia Universidad Católica del Perú
* Sección Ingeniería Mecatrónica

---
Este archivo describe el método de control cinemático para robots manipuladores
a partir de su matriz Jacobiana empleando un simple control proporcional.
Se realiza un análisis y derivación de la ley de control seguido por una simulación en python donde se aprecia el desempeño del sistema de control.

<img src="https://drive.google.com/uc?export=view&id=1vgFoFHEKMN0WcAFlKS2hPMg2KuBm3sp9" alt="drawing"/>

#Control Cinemático

## Definición

El objetivo del Control Cinemático es definir una ley de control (ecuación matemática) en retroalimentación de estados. Siendo las posiciones de las articulaciones, el estado del robot.

Esta ley proporciona una expresión de velocidades que deben de tener las articulaciones para que el efector final siga una trayectoria y/o posicion definida.

Este tipo de control emplea la cinemática directa y la jacobiana del manipualdor para cálcular dicha ley de control.

## Derivación de ley de control

Considerar el manipulador de 2 grados de libertad mostrado en la siguiente figura. El manipulador se encuentra en una posición inicial definida por el vector $x$ y se desea que se mueva a la nueva posición marcada con una estrella verde cuya posición se define por $x_{ref}$.

<img src="https://drive.google.com/uc?export=view&id=1gWt9XDmUWNyXIbT9Q3UV8CEX7Vg7UFsk" alt="Control Cinematico Inicial" width=200/>

A partir de estas dos posiciones, se puede analizar definir una función de describa la diferencia entre la posición actual del robot y la posición deseada - es decir, se puede definir una función de error.

$e_{rr} = x_{ref} - x$

### Definición de funciones auxiliares de cinemática directa y jacobiano

In [21]:
import numpy as np

# =============================================================================
# Se definen una función para calcular la transformación de Denavit-Hartenberg
# =============================================================================
def transformacion_DH_num(d: float, a: float, alfa: float, theta: float) -> np.ndarray:
  """
  Retorna una representación numérica de la transformación de Denavit-Hartenberg

  Parametros (float):
    d    : Traslación 𝑑 a lo largo del eje Z
    a    : Traslación 𝑎 a lo largo del eje X
    theta: Rotación 𝜃 a lo largo del eje Z
    alfa : Rotación 𝛼 a lo largo del eje X
  """
  cos_theta = np.cos(theta)
  sin_theta = np.sin(theta)
  cos_alfa  = np.cos(alfa)
  sin_alfa  = np.sin(alfa)
  T = np.array([[cos_theta, -cos_alfa*sin_theta,  sin_alfa*sin_theta, a*cos_theta],
                [sin_theta,  cos_alfa*cos_theta, -sin_alfa*cos_theta, a*sin_theta],
                [0, sin_alfa, cos_alfa, d],
                [0, 0, 0, 1]])
  return T

# =============================================================================
# Se define la función para calcular la cinemática directa del manipulador de
# 2 grados de libertad RR
# =============================================================================
def cinematica_directa_robot2D(q1: float, q2: float, l1: float, l2: float):
  """
  Calcula la cinemática directa del robot 2D

  Parámetros (float):
      q1: Ángulo de la articulación 1 en radianes.
      q2: Ángulo de la articulación 2 en radianes.
      l1: Longitud del eslabón 1.
      l2: Longitud del eslabón 2.

  Retorna:
      Una tupla de matriz 4x4 que representan las transformaciones homogénea de
      cada eslabón con respecto a la base del robot.
  """
  # Se definen las matrices de transformacion homogenea correspondientes empleando
  # la notación:
  #    Tab: Transformación del sistema 'b' con respecto al sistema 'a'
  T01 = transformacion_DH_num(0,l1,0,q1)
  T12 = transformacion_DH_num(0,l2,0,q2)
  T02 = T01 @ T12
  return T01, T02

In [22]:
# =============================================================================
# Se define la función para calcular el jacobiano del manipulador de 2 grados
# de libertad RR
# =============================================================================
def jacobiano_robot2D(q1: float, q2: float, l1: float, l2: float):
  """
  Calcula el jacobiano del robot 2D

  Parámetros (float):
      q1: Ángulo de la articulación 1 en radianes.
      q2: Ángulo de la articulación 2 en radianes.
      l1: Longitud del eslabón 1.
      l2: Longitud del eslabón 2.

  Retorna:
      Una matriz 2x2 que representa el Jacobiano del robot.
  """
  J = np.array([[-l1 * np.sin(q1) - l2 * np.sin(q1 + q2), -l2 * np.sin(q1 + q2)],
                [l1 * np.cos(q1) + l2 * np.cos(q1 + q2), l2 * np.cos(q1 + q2)]])
  return J


### Función auxiliar para gráficar y realizar la simulación

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc

plt.rcParams['axes.grid'] = True

class Robot_2D_animacion():
  '''
  Clase para animar el movimiento de un manipulador robótico 2D en tiempo real.

  Atributos:
    fig: Objeto de la figura para la animación.
    ax: arreglo de subfiguras para graficar diferentes componentes de la animación.
    tt: Serie de datos de tiempo.
    qq: Serie de datos de ángulo de las articulaciones.
    ee: Serie de datos de error.
    xx: Serie de datos de posición del efector final.
  '''
  # Inicializa la figura y los subplots para la animación.
  fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 6), tight_layout=True)
  ax[0][0].set_aspect('equal')
  plt.close()

  # Crea gráficos para los componentes de la animación.
  line, = ax[0,0].plot([], [], 'o-', color='purple', lw=2) # Línea del eslabón
  joint, = ax[0,0].plot([], [], 'o', color='y', markersize=5) # Posición de la articulación
  trayectory, = ax[0,0].plot([], [], ':', color='r',markersize=1) # Trayectoria del efector final
  deseado, = ax[0,0].plot([], [], '*', color='g',markersize=5)

  articulaciones = ax[1,0].plot([],[], [], [], [2.5], [0], lw=1) # Posiciones de las articulaciones
  posicion_x, = ax[0,1].plot([], [], lw=1) # Posición X del efector final
  posicion_y, = ax[0,1].plot([], [], lw=1) # Posición Y del efector final
  posicion_x_sp, = ax[0,1].plot([], [], 'g--', lw=1) # Posición X deseada
  posicion_y_sp, = ax[0,1].plot([], [], 'g--', lw=1) # Posición Y deseada
  error_x, = ax[1,1].plot([], [], lw=1) # Error en X
  error_y, = ax[1,1].plot([], [], lw=1) # Error en Y
  error_sp, = ax[1,1].plot([], [], 'g--', lw=1) # Error deseado (cero)

  # Pre-inicialización para las series datos a emplear
  tt = []
  qq = []
  ee = []
  xx = []

  @classmethod
  def reset_plot(cls):
    '''
    Reinicia los datos almacenados para la animación limpiando las listas de
    tiempo, ángulo de las articulaciones, error y posición del efector final.
    También establece un valor de error alto arbitrario.
    '''
    cls.tt.clear()
    cls.qq.clear()
    cls.ee.clear()
    cls.xx.clear()
    cls.e = 10000 # Arbitrary high error value

  @classmethod
  def configure_plot(cls, pos_deseada, time_end=2.5, epsilon=1e-1, max_iter=100):
    '''
    Configuración de los gráficos, incluyendo los límites para los ejes, leyendas y títulos

    Parámetros:
      pos_deseada (array-like): Posición deseada del efector final.
      time_end (float): El tiempo final para la simulación.
      epsilon (float): Tolerancia para el error, utilizado para determinar la convergencia.
      max_iter (int): Número máximo de iteraciones para la simulación.
    '''
    cls.reset_plot()
    cls.pos_deseada = pos_deseada
    cls.epsilon = epsilon
    cls.max_iter = max_iter
    cls.time_end = time_end

    # Establecer límites de los ejes y titulo para cada subgráfica
    cls.ax[0,0].set_title('Simulación del manipulador')
    cls.ax[0,0].set_xlim((-15, 15))
    cls.ax[0,0].set_ylim((-15, 15))
    cls.ax[0,0].grid(False)

    cls.ax[0,1].set_title('Posición del efector final')
    cls.ax[0,1].set_ylim((1.5 * min(pos_deseada), 1.5 * max(pos_deseada)))
    cls.ax[0,1].legend(['Coordenada X', 'Coordenada Y'])
    cls.ax[0,1].set_xlabel('Time')
    cls.ax[0,1].set_ylabel('Posición')

    cls.ax[1,0].set_title('Posición de las articulaciones')
    cls.ax[1,0].set_ylim(( -180, 180))
    cls.ax[1,0].legend(['q1', 'q2'])
    cls.ax[1,0].set_xlabel('Time')
    cls.ax[1,0].set_ylabel('Posición (°)')

    cls.ax[1,1].set_title('Errores')
    cls.ax[1,1].set_ylim(( -20, 20))
    cls.ax[1,1].legend(['Error en X', 'Error en Y'])
    cls.ax[1,1].set_xlabel('Time')
    cls.ax[1,1].set_ylabel('Posición')

    cls.deseado.set_data(pos_deseada[0], pos_deseada[1])

  @classmethod
  def update_plot(cls, t, q, e, x1, y1, x2, y2):
    '''
    Actualiza el gráfico con los nuevos datos en cada frame de la animación.

    Parámetros:
      t (float): Tiempo actual.
      q (array-like): Ángulos actuales de las articulaciones.
      e (array-like): Valores actuales de error.
      x1 (float): Coordenada X actual de la primera articulación.
      y1 (float): Coordenada Y actual de la primera articulación.
      x2 (float): Coordenada X actual del efector final.
      y2 (float): Coordenada Y actual del efector final.
    '''
    cls.tt.append(t)
    cls.qq.append(np.rad2deg(q))
    cls.ee.append(e)
    cls.xx.append([x2, y2])
    cls.e = e

    # Direct array updates for efficiency
    q_array = np.array(cls.qq)
    x_array = np.array(cls.xx)
    e_array = np.array(cls.ee)

    cls.articulaciones[0].set_data(cls.tt, q_array[:,0])
    cls.articulaciones[1].set_data(cls.tt, q_array[:,1])

    cls.posicion_x.set_data(cls.tt, x_array[:,0])
    cls.posicion_y.set_data(cls.tt, x_array[:,1])

    cls.error_x.set_data(cls.tt, e_array[:,0])
    cls.error_y.set_data(cls.tt, e_array[:,1])

    cls.line.set_data([0, x1, x2], [0, y1, y2])
    cls.joint.set_data([0, x1], [0, y1])
    cls.trayectory.set_data(x_array.T)

    # auto escala las gráficas cuando el tiempo es 0 o supera time_end (2.5)
    if t==0 or t>cls.time_end:
      t_end = max(t,cls.time_end)
      cls.posicion_x_sp.set_data([0,t_end], [cls.pos_deseada[0],cls.pos_deseada[0]])
      cls.posicion_y_sp.set_data([0,t_end], [cls.pos_deseada[1],cls.pos_deseada[1]])
      cls.error_sp.set_data([0,t_end], [0,0])
      # recompute the ax.dataLim
      cls.ax[0,1].relim()
      cls.ax[1,0].relim()
      cls.ax[1,1].relim()
      # update ax.viewLim using the new dataLim
      cls.ax[0,1].autoscale_view()
      cls.ax[1,0].autoscale_view()
      cls.ax[1,1].autoscale_view()

  @staticmethod
  def gen():
    '''
    Genera frames para la animación mientras el error esté por encima del umbral
    especificado y no se haya alcanzado el número máximo de iteraciones.

    Yields:
      int: El número de la iteración actual.
    '''
    i = 0
    while np.linalg.norm(Robot_2D_animacion.e) > Robot_2D_animacion.epsilon and i < Robot_2D_animacion.max_iter:
        i += 1
        yield i
    if i == Robot_2D_animacion.max_iter:
      print('Se alcanzó el número máximo de iteraciones')
    else:
      print(f'Simulación terminó en {i} iteraciones con un error máximo de {np.linalg.norm(Robot_2D_animacion.e):.2f}')
      print(f'Robot alcanza ubicación {np.around(Robot_2D_animacion.xx[-1],2)} en {Robot_2D_animacion.tt[-1]:.2f} segundos')

  @classmethod
  def init(cls):
    '''
    Inicializa la animación estableciendo los datos de la línea como vacíos.

    Returns:
      tuple: Una tupla que contiene el objeto de línea inicializado.
    '''
    cls.line.set_data([], [])
    return (cls.line,)

  @classmethod
  def run_animation(cls, animate):
    """
    Ejecuta la animación utilizando FuncAnimation de matplotlib.

    Parámetros:
      animate (function): La función de animación que se llamará para cada fotograma.

    Returns:
      anim: El objeto de animación creado por FuncAnimation.
    """
    anim = animation.FuncAnimation(cls.fig, animate, init_func=cls.init,
                                   frames=cls.gen, interval=100, blit=True, repeat=False,
                                   cache_frame_data=False)

    # Nota: la siguiente parte es necesaria para ejecutar el código en colab
    rc('animation', html='jshtml')
    return anim

### Simulación de Control Cinemático de un manipulador

El siguiente código ejecuta el control de un manipulador para trasladarse desde una configuración inicial hacia una posición final a una velocidad predeterminada.

Los valores por defecto están dados según la siguiente configuración
```
# Posición y velocidades deseadas del efector final
pos_deseada = np.array([-8, 12])
vel_deseada = np.array([-1, 0])

# Configuración articular inicial del robot
q = [np.deg2rad(45), np.deg2rad(25)]
# Constante de ganancia cinemática
k = np.diag([1,1]) #[1,1]
```



In [29]:
import numpy as np
import time

# Variables temporales para la simulación
t = 0
dt = 0.05
# Longitudes de los eslabones del manipulador
l1 = 10
l2 = 5
# Posición y velocidades deseadas del efector final
pos_deseada = np.array([-8, 12]) #[-8, 12]
vel_deseada = np.array([-1, 0]) #[-1, 0]

# Configuración articular inicial del robot
q = [np.deg2rad(45), np.deg2rad(25)] #np.deg2rad(45), np.deg2rad(25)
# Constante de ganancia cinemática
k = np.diag([1,1]) #[1,1]

# La simulación puede tomar hasta 1 minuto y medio. Si se desea que ejecute más
# rápido considerar limitar la cantidad de iteraciones máximas o
Robot_2D_animacion.configure_plot(pos_deseada, epsilon=1e0, max_iter=100)

# =============================================================================
# animation function. This is called sequentially
# =============================================================================
def animate(i):

    global q
    global t

    # Cálculo de Jacobiano y cinemática directa para la posición actual del manipulador
    q1 = q[0]
    q2 = q[1]
    J = jacobiano_robot2D(q1, q2, l1, l2)
    T01, T02 = cinematica_directa_robot2D(q1, q2, l1, l2)

    # Obtener las coordenadas del codo
    x1 = T01[0, 3]
    y1 = T01[1, 3]
    # Obtener las coordenadas del efector final
    x2 = T02[0, 3]
    y2 = T02[1, 3]
    pos_actual = [x2, y2]

    # Error en la posición
    e = pos_deseada-pos_actual
    # Aplicación de la ley de control (Recordar que los cálculos son matriciales)
    dq = np.linalg.pinv(J) @ (vel_deseada + k @ e)
    # Integración de Euler para obtener la nueva configuración articular
    q = q + dt*dq

    Robot_2D_animacion.update_plot(t,q,e,x1,y1,x2,y2)
    # Actualizar el tiempo
    t = t + dt
    return (Robot_2D_animacion.line,)

anim = Robot_2D_animacion.run_animation(animate)
anim

Output hidden; open in https://colab.research.google.com to view.